# Python WTF

虽说 Python 是一个相对来说设计比较完善的编程语言，但是其中仍有许多让人费解甚至哭笑不得的地方。
[Reddit 上](https://www.reddit.com/r/Python/comments/3cu6ej/what_are_some_wtf_things_about_python/)就有一个题为“Python WTF”的话题，这里搬运过来供大家引以为戒！

## ``datatime.time`` 中的陷阱

[ref-link](https://www.reddit.com/r/Python/comments/3cu6ej/what_are_some_wtf_things_about_python/csz47mj)

In [1]:
import datetime

mid_night = datetime.time(0)
noon = datetime.time(12)

if mid_night:
    print True
else:
    print 'mid_night represents False.'
    
if noon:
    print 'noon represents True.'

mid_night represents False.
noon represents True.


虽然很反直觉，但是一般并没有人实用 ``datetime.time`` 做真假校验吧？

PS：[Python 3.5](https://docs.python.org/3.5/whatsnew/3.5.html )
中已修正。

Reddit 网友点评：

![Who let the PHP design committee in here!?](images/datetime.png)

## 257 是一个神奇的数字

[ref-link](https://www.reddit.com/r/Python/comments/3cu6ej/what_are_some_wtf_things_about_python/csz0vwv)

In [2]:
a = 256
b = 256
print a is b

c = 257
d = 257
print c is d
print c == d

True
False
True


我们说 Python 中一切皆对象，所有数字、字符串、类、函数都是对象，而变量只是对象的引用。
也许你还知道，在 Python 中 ``==`` 的实现取决于对象的魔术方法 ``__eq__`` ，
而 ``is`` 是否正确取决于两个变量的内存地址是否相同，也就是说在上面的代码中，
``c`` 和 ``d`` 的地址并不一致。

In [3]:
a = [1, 2, 3]
print id(a)

a.append(4)
print id(a)

a = 3
print id(a)

a = 4
print id(a)

a = 'kxxoling'
print id(a)

a = 'Kane'
print id(a)

4582841088
4582841088
140588812894040
140588812894016
4587313408
4587314368


实际上，可能是出于性能考虑，Python 中 -5 ~ 256 范围内的整数以及少部分字符串对象在程序启动时已经存在于内存中，并且不会 GC 被回收。
因此，无论何时声明变量为其中的值，变量的指针都指向同一个内存地址。

PS：上面的问题只会在某些 REPL 环境下出现，我当前的 Python 版本是 2.7.10 （OS X 10.10）。

## 循环中的函数定义

作为一名 Python 程序员，虽然很喜欢 Python 的函数编程特性，但不得不承认 Python 对于函数式编程的支持甚至不如没有一类函数对象的 Ruby。
对于函数式编程语言来说，惰性计算是一个非常重要的概念，而 Python 相对来说是一个非常“急性子”的编程语言。

思考一下下面这段代码：

In [4]:
func_list = [lambda x: x**i for i in range(10)]
print [f(2) for f in func_list]

[512, 512, 512, 512, 512, 512, 512, 512, 512, 512]


实际上这段代码和下面这段是等效的：

In [5]:
func_list = []
lamb = None
i = 0
lst = []
for _ in range(10):
    i = _
    lst.append(i)

print i
for i_ in lst:
    lamb = lambda x: x**i
    func_list.append(lamb)

result = []
for f in func_list:
    result.append(f(2))

print result

9
[512, 512, 512, 512, 512, 512, 512, 512, 512, 512]


我们稍微修改一下看看：

In [6]:
func_list = []
lamb = None
for i in range(10):
    lamb = lambda x, i=i: x**i     # 注意这里最后一个 i，根本的变化是这里的 i 是函数的内部变量而非外部。
    func_list.append(lamb)

result = []
for f in func_list:
    result.append(f(2))

print result

[1, 2, 4, 8, 16, 32, 64, 128, 256, 512]


结果已经和我们期待的一样了。

## 元组解析

Python 中这种语法被称为元组解析：

In [7]:
a, b = (1, 2)

(a, b) = (1, 2)
[a, b] = (1, 2)
[a, b] = [1, 2]

不清楚元组解析的实现，
[Reddit 网友](https://www.reddit.com/r/Python/comments/3cu6ej/what_are_some_wtf_things_about_python/csz218x)
给出的解释是这样的：

In [8]:
some_iterable = [1, 2]
_iterator = iter(some_iterable)
a = next(_iterator)
b = next(_iterator)
for superfluous_element in _iterator:
    # this only happens if there’s something left
    raise('Expected some_iterable to have 2 elements')

因此我们可以写出这样无意义的代码：

In [9]:
[] = ()

但是等号左右反过来的话，Python 就不会将其当作元组解析对待了。

# True is False == False ？

看看下面的执行结果为什么不同：

In [10]:
print True is False == False
print (True is False) == False
print True is (False == False)

False
True
True


第一行的代码必然是相当于第二行或者第三行？在 Python 的实现中并非如此：

In [11]:
import dis
dis.dis(lambda: True is False == False)

  2           0 LOAD_GLOBAL              0 (True)
              3 LOAD_GLOBAL              1 (False)
              6 DUP_TOP             
              7 ROT_THREE           
              8 COMPARE_OP               8 (is)
             11 JUMP_IF_FALSE_OR_POP    21
             14 LOAD_GLOBAL              1 (False)
             17 COMPARE_OP               2 (==)
             20 RETURN_VALUE        
        >>   21 ROT_TWO             
             22 POP_TOP             
             23 RETURN_VALUE        


实际上 ``True is False == False`` 相当于 ``(True is False) and (False == False)`` ：

In [12]:
(True is False) and (False == False)

False